# En este documento se describen las TRANSFORMACIONES hechas al dataset movies y credits necesarias y previas al desarrollo y conclusion del Mminimum Viable Product

Inicamos con importar las librerias que nos seran de utilidad a tales efectos.

In [132]:
import pandas as pd
import numpy as np

Esta libreria en particular sera para desanidar algunas columnas como veremos mas adelante:

In [133]:
import json
from ast import literal_eval 

# Las transformaciones necesrias comienzan con desanidar columnas con datos en forma de lista de diccionario o diccionario en los datasets de movies y credits.

# Comenzamos con el datasets de credits:

In [134]:
credits = pd.read_csv("credits.csv")
credits.rename(columns = {'id':'pelicula_id'}, inplace = True)
credits.head()

,cast,crew,pelicula_id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Luego de Cargar el .csv, y renombrar la columna id, siendo las columnas cast y crew listas de diccionarios, hay que desanidarlas. Se se realiza en 4 pasos, comenzando con la columna cast:

In [135]:
#1
credits_cast=pd.DataFrame(credits[['cast','pelicula_id']])
credits_cast['cast']=credits_cast['cast'].apply(literal_eval)
credits_cast.head()

,cast,pelicula_id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",15602
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...",11862


In [136]:
#2
credits_cast_exploded = credits_cast.explode('cast', ignore_index= True) 
credits_cast_exploded.head()

,cast,pelicula_id
0,"{'cast_id': 14, 'character': 'Woody (voice)', ...",862
1,"{'cast_id': 15, 'character': 'Buzz Lightyear (...",862
2,"{'cast_id': 16, 'character': 'Mr. Potato Head ...",862
3,"{'cast_id': 17, 'character': 'Slinky Dog (voic...",862
4,"{'cast_id': 18, 'character': 'Rex (voice)', 'c...",862


In [137]:
#3
credits_cast_exploded = credits_cast_exploded.set_index('pelicula_id')
credits_cast_normalize = pd.json_normalize(credits_cast_exploded['cast']).set_index(credits_cast_exploded.index)
credits_cast_normalize.head()

,cast_id,character,credit_id,gender,id,name,order,profile_path
pelicula_id,,,,,,,,
862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


In [138]:
credits_cast_normalize.rename(columns={'name' : 'nombre', 'character': 'presonaje'}, inplace= True)

In [139]:
credits_cast_normalize.head()

,cast_id,presonaje,credit_id,gender,id,nombre,order,profile_path
pelicula_id,,,,,,,,
862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


In [140]:
#4
credits_cast_normalize.reset_index(inplace=True)
credits_cast_normalize.head()

,pelicula_id,cast_id,presonaje,credit_id,gender,id,nombre,order,profile_path
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


Los pasos para desanidar la lista de direccionario de la culumna cast fueron los siguientes:
#1: creo un df solo para la columna crast y pelicula_id. Y luego aplico a cada registro el metodo literal_eval. Este metodo normaliza la sintaxis para el tipo de objeto partiuclar del que se trate. En este caso
es un json. El resultado no parecera ser distinto a los registros previos, pero la sintaxis ha sido etiquetada como json y permitira aplicar finalmente el metodo del #3. 

#2: en este paso, 'explotamos' cada elemento de la lista en cada registro en la columna cast y la incluimos como un registro indempendiente. Con ignore_index = True, nos aceguramos que indice de las filas sea secuencia y no tenga en cuenta el indice original.

#3: aqui "normalizamos" los objetos json generando tantas columnas como claves en los diccionarios, y en cada registro el valor que dicha clave tiene en la linea. Vale aclarar que en um principio obien el set.index,
para luego contatenar horizontalmente con credicts agregando el pelicula_id. El problema fue que ambas tablas no tenian el mismo numero de registros y no coincidian dichos peliculas_id. Por lo que setee el index y luego pase al punto #4.

#4: aqui reseteo el indice de cretis_cast_normalize agregando como indice otros distintos al pelicula_id.

# Ahora hacemos lo mismo con la columna crew:

In [141]:
credits_crew=pd.DataFrame(credits[['crew','pelicula_id']])
credits_crew.head(3)


,crew,pelicula_id
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [142]:
credits_crew['crew']=credits_crew['crew'].apply(literal_eval)
credits_crew.head(3)


,crew,pelicula_id
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [143]:
credits_crew_exploded = credits_crew.explode('crew', ignore_index=True)
credits_crew_exploded.head(3)


,crew,pelicula_id
0,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep...",862
1,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep...",862
2,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep...",862


In [144]:
credits_crew_exploded = credits_crew_exploded.set_index('pelicula_id')
credits_crew_exploded.head(3)


,crew
pelicula_id,
862,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep..."
862,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep..."
862,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep..."


In [145]:
credits_crew_normalize = pd.json_normalize(credits_crew_exploded['crew']).set_index(credits_crew_exploded.index)
credits_crew_normalize.head(3)

,credit_id,department,gender,id,job,name,profile_path
pelicula_id,,,,,,,
862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg


In [146]:
credits_crew_normalize.reset_index(inplace=True)
credits_crew_normalize.head()

,pelicula_id,credit_id,department,gender,id,job,name,profile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [147]:
credits_crew_normalize.rename(columns = {'department' : 'crew_area', 'job' : 'puesto', 'name' : 'nombre'}, inplace=True)
credits_crew_normalize.head()

,pelicula_id,credit_id,crew_area,gender,id,puesto,nombre,profile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


Una vez normalizados los df credits_cast y credicts_crew, hay que unirlas. Pero antes debemos renombrar algunas columnas omonimas y eliminar otaas redundantes.

In [148]:
credits_cast_normalize.rename(columns = {'nombre': 'cast_nombre', 'id':'id01'}, inplace=True)
credits_cast_normalize.drop(['profile_path', 'credit_id', 'gender'], axis=1, inplace=True)
credits_cast_normalize.head()

,pelicula_id,cast_id,presonaje,id01,cast_nombre,order
0,862,14.0,Woody (voice),31.0,Tom Hanks,0.0
1,862,15.0,Buzz Lightyear (voice),12898.0,Tim Allen,1.0
2,862,16.0,Mr. Potato Head (voice),7167.0,Don Rickles,2.0
3,862,17.0,Slinky Dog (voice),12899.0,Jim Varney,3.0
4,862,18.0,Rex (voice),12900.0,Wallace Shawn,4.0


In [149]:
credits_crew_normalize.rename(columns = {'nombre': 'crew_nombre', 'id':'id02'}, inplace=True)
credits_crew_normalize.drop(['profile_path', 'credit_id'], axis=1, inplace=True)
credits_crew_normalize.head()

,pelicula_id,crew_area,gender,id02,puesto,crew_nombre
0,862,Directing,2.0,7879.0,Director,John Lasseter
1,862,Writing,2.0,12891.0,Screenplay,Joss Whedon
2,862,Writing,2.0,7.0,Screenplay,Andrew Stanton
3,862,Writing,2.0,12892.0,Screenplay,Joel Cohen
4,862,Writing,0.0,12893.0,Screenplay,Alec Sokolow


In [150]:
credit_normalizado = pd.merge(credits_cast_normalize, credits_crew_normalize, on = 'pelicula_id')

In [151]:
credit_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9495072 entries, 0 to 9495071
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   pelicula_id  int64  
 1   cast_id      float64
 2   presonaje    object 
 3   id01         float64
 4   cast_nombre  object 
 5   order        float64
 6   crew_area    object 
 7   gender       float64
 8   id02         float64
 9   puesto       object 
 10  crew_nombre  object 
dtypes: float64(5), int64(1), object(5)
memory usage: 869.3+ MB


In [152]:
credit_duplicadas = credit_normalizado.duplicated().sum()
print(credit_duplicadas)

15586


# Dado que hay registros duplicados habria que eliminarlos:

In [153]:
credit_normalizado = credit_normalizado.drop_duplicates()
credit_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9479486 entries, 0 to 9495071
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   pelicula_id  int64  
 1   cast_id      float64
 2   presonaje    object 
 3   id01         float64
 4   cast_nombre  object 
 5   order        float64
 6   crew_area    object 
 7   gender       float64
 8   id02         float64
 9   puesto       object 
 10  crew_nombre  object 
dtypes: float64(5), int64(1), object(5)
memory usage: 867.9+ MB


# Ahora resta pasar este dataset a un .csv:

In [154]:
credit_normalizado.to_csv('credit_normalizado.csv')

# DATASET MOVIES.

Cargando el datasets movies, vemos que hay nuevamente registros con datos anidados. A diferencia del ds credits, no son listas de diccionario, sino solo diccionarios, por lo que no seria 
necesario aplicar el motodo explode.

In [155]:
movies = pd.read_csv('movies_dataset.csv')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

C:\Users\alfon\AppData\Local\Temp\ipykernel_13664\1532156847.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_dataset.csv')


# Columana belongs_to_collection

In [156]:
movies_collection = pd.DataFrame(movies[['belongs_to_collection', 'id']]).dropna()
movies_collection.rename(columns= {'id' : 'pelicula_id'}, inplace= True)
movies_collection.head()

,belongs_to_collection,pelicula_id
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",862
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",15602
4,"{'id': 96871, 'name': 'Father of the Bride Col...",11862
9,"{'id': 645, 'name': 'James Bond Collection', '...",710
12,"{'id': 117693, 'name': 'Balto Collection', 'po...",21032


In [157]:
movies_collection['belongs_to_collection'] = movies_collection['belongs_to_collection'].apply(literal_eval)
movies_collection.head()

,belongs_to_collection,pelicula_id
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",862
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",15602
4,"{'id': 96871, 'name': 'Father of the Bride Col...",11862
9,"{'id': 645, 'name': 'James Bond Collection', '...",710
12,"{'id': 117693, 'name': 'Balto Collection', 'po...",21032


En este caso se obvia el explode ya que como se ve los registros de la columna belongs_to_collection ya son diccionarios, no listas. Por lo que se puede pasar dicectamente
a la normalizacion.

In [158]:
movies_collection = movies_collection.set_index('pelicula_id')
movies_collection_normalize = pd.json_normalize(movies_collection['belongs_to_collection']).set_index(movies_collection.index)

movies_collection_normalize.head()

,id,name,poster_path,backdrop_path
pelicula_id,,,,
862,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
15602,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
11862,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
710,645.0,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg
21032,117693.0,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg


In [159]:
movies_collection_normalize.reset_index(inplace=True)
movies_collection_normalize.head()

,pelicula_id,id,name,poster_path,backdrop_path
0,862,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,15602,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
2,11862,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
3,710,645.0,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg
4,21032,117693.0,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg


In [160]:
movies_collection_normalize.rename(columns={'id' : 'franquicia_id', 'name':'franquicia'}, inplace= True)
movies_collection_normalize.drop({'poster_path', 'backdrop_path'}, axis=1, inplace=True)
movies_collection_normalize.head(2)

,pelicula_id,franquicia_id,franquicia
0,862,10194.0,Toy Story Collection
1,15602,119050.0,Grumpy Old Men Collection


# AHORA VAMOS CON LA COLUMNA GENRES (generos)

Como vemos aqui, genres debe ser explotadas ya que no es un diccionario sino una lista

In [161]:
movies_genres = pd.DataFrame(movies[['genres', 'id']]).dropna()
movies_genres.rename(columns= {'id' : 'pelicula_id'}, inplace= True)
movies_genres.head(3)

,genres,pelicula_id
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602


In [162]:
movies_genres['genres'] = movies_genres['genres'].apply(literal_eval)

In [163]:
movies_genres_exploded = movies_genres.explode('genres', ignore_index= True)
movies_genres_exploded.head(3)

,genres,pelicula_id
0,"{'id': 16, 'name': 'Animation'}",862
1,"{'id': 35, 'name': 'Comedy'}",862
2,"{'id': 10751, 'name': 'Family'}",862


In [164]:
movies_genres_exploded = movies_genres_exploded.set_index('pelicula_id')
movies_genres_exploded.head(3)

,genres
pelicula_id,
862,"{'id': 16, 'name': 'Animation'}"
862,"{'id': 35, 'name': 'Comedy'}"
862,"{'id': 10751, 'name': 'Family'}"


In [165]:
movies_genres_normalize = pd.json_normalize(movies_genres_exploded['genres']).set_index(movies_genres_exploded.index)
movies_genres_normalize.head(3)

,id,name
pelicula_id,,
862,16.0,Animation
862,35.0,Comedy
862,10751.0,Family


In [166]:
movies_genres_normalize.reset_index(inplace=True)
movies_genres_normalize.head(3)

,pelicula_id,id,name
0,862,16.0,Animation
1,862,35.0,Comedy
2,862,10751.0,Family


In [167]:
movies_genres_normalize.rename(columns= {'id' : 'genero_id', 'name':'nombre_genero'}, inplace=True)
movies_genres_normalize.head(2)

,pelicula_id,genero_id,nombre_genero
0,862,16.0,Animation
1,862,35.0,Comedy


# AHORA VAMOS CON LA COLUMNA production_companies

In [168]:
movies_production = pd.DataFrame(movies[['production_companies', 'id']]).dropna()
movies_production.head(3)

,production_companies,id
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]",862
1,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",8844
2,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",15602


In [169]:
movies_production.rename(columns= {'id' : 'pelicula_id'}, inplace= True)
movies_production.head(2)

,production_companies,pelicula_id
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]",862
1,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",8844


In [170]:
movies_production['production_companies'] = movies_production['production_companies'].apply(literal_eval)

In [171]:
movies_production_explode = movies_production.explode('production_companies', ignore_index= True)
movies_production_explode.head()

,production_companies,pelicula_id
0,"{'name': 'Pixar Animation Studios', 'id': 3}",862
1,"{'name': 'TriStar Pictures', 'id': 559}",8844
2,"{'name': 'Teitler Film', 'id': 2550}",8844
3,"{'name': 'Interscope Communications', 'id': 10...",8844
4,"{'name': 'Warner Bros.', 'id': 6194}",15602


In [172]:
movies_production_normalize = pd.json_normalize(movies_production_explode['production_companies']).set_index(movies_production_explode.pelicula_id)

In [173]:
movies_production_normalize.head(3)

,name,id
pelicula_id,,
862,Pixar Animation Studios,3.0
8844,TriStar Pictures,559.0
8844,Teitler Film,2550.0


In [174]:
movies_production_normalize.reset_index(inplace=True)
movies_production_normalize.head(3)


,pelicula_id,name,id
0,862,Pixar Animation Studios,3.0
1,8844,TriStar Pictures,559.0
2,8844,Teitler Film,2550.0


In [175]:
movies_production_normalize.rename(columns={'name' : 'compañia'}, inplace= True)
movies_production_normalize.rename(columns={'id' : 'compañia_id'}, inplace=True)
movies_production_normalize.head(2)

,pelicula_id,compañia,compañia_id
0,862,Pixar Animation Studios,3.0
1,8844,TriStar Pictures,559.0


# AHORA VAMOS POR production_countries

In [176]:
movies_countries = pd.DataFrame(movies[['production_countries', 'id']]).dropna()
movies_countries.head()

,production_countries,id
0,"[{'iso_3166_1': 'US', 'name': 'United States o...",862
1,"[{'iso_3166_1': 'US', 'name': 'United States o...",8844
2,"[{'iso_3166_1': 'US', 'name': 'United States o...",15602
3,"[{'iso_3166_1': 'US', 'name': 'United States o...",31357
4,"[{'iso_3166_1': 'US', 'name': 'United States o...",11862


In [177]:
movies_countries['production_countries'] = movies_countries['production_countries'].apply(literal_eval)
movies_countries.head(3)

,production_countries,id
0,"[{'iso_3166_1': 'US', 'name': 'United States o...",862
1,"[{'iso_3166_1': 'US', 'name': 'United States o...",8844
2,"[{'iso_3166_1': 'US', 'name': 'United States o...",15602


In [178]:
# Por una omision de mi parte de hacerlo antes, recien ahora renombro la columna id:

movies_countries.rename(columns= {'id' : 'pelicula_id'}, inplace=True)
movies_countries.head(3)

,production_countries,pelicula_id
0,"[{'iso_3166_1': 'US', 'name': 'United States o...",862
1,"[{'iso_3166_1': 'US', 'name': 'United States o...",8844
2,"[{'iso_3166_1': 'US', 'name': 'United States o...",15602


In [179]:
movies_countries_explode = movies_countries.explode('production_countries')
movies_countries_explode.head(3)

,production_countries,pelicula_id
0,"{'iso_3166_1': 'US', 'name': 'United States of...",862
1,"{'iso_3166_1': 'US', 'name': 'United States of...",8844
2,"{'iso_3166_1': 'US', 'name': 'United States of...",15602


In [180]:
movies_countries_normalize = pd.json_normalize(movies_countries_explode['production_countries']).set_index(movies_countries_explode.pelicula_id)
movies_countries_normalize.head()

,iso_3166_1,name
pelicula_id,,
862,US,United States of America
8844,US,United States of America
15602,US,United States of America
31357,US,United States of America
11862,US,United States of America


In [181]:
movies_countries_normalize.reset_index(inplace=True)
movies_countries_normalize.head()

,pelicula_id,iso_3166_1,name
0,862,US,United States of America
1,8844,US,United States of America
2,15602,US,United States of America
3,31357,US,United States of America
4,11862,US,United States of America


In [182]:
movies_countries_normalize.rename(columns={'name' : 'pais'}, inplace=True)
movies_countries_normalize.head(3)

,pelicula_id,iso_3166_1,pais
0,862,US,United States of America
1,8844,US,United States of America
2,15602,US,United States of America


# VAMOS POR spkeN_languages

In [183]:
movies_languages = pd.DataFrame(movies[['spoken_languages', 'id']]).dropna()
movies_languages.rename(columns= {'id' : 'pelicula_id'}, inplace=True)
movies_languages.head()

,spoken_languages,pelicula_id
0,"[{'iso_639_1': 'en', 'name': 'English'}]",862
1,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",8844
2,"[{'iso_639_1': 'en', 'name': 'English'}]",15602
3,"[{'iso_639_1': 'en', 'name': 'English'}]",31357
4,"[{'iso_639_1': 'en', 'name': 'English'}]",11862


In [184]:
movies_languages['spoken_languages']= movies_languages['spoken_languages'].apply(literal_eval)

In [185]:
movies_languages_explode = movies_languages.explode('spoken_languages')
movies_languages_explode.head()

,spoken_languages,pelicula_id
0,"{'iso_639_1': 'en', 'name': 'English'}",862
1,"{'iso_639_1': 'en', 'name': 'English'}",8844
1,"{'iso_639_1': 'fr', 'name': 'Français'}",8844
2,"{'iso_639_1': 'en', 'name': 'English'}",15602
3,"{'iso_639_1': 'en', 'name': 'English'}",31357


In [186]:
movies_languages_normalize = pd.json_normalize(movies_languages_explode['spoken_languages']).set_index(movies_languages_explode.pelicula_id)

In [187]:
movies_languages_normalize.head()

,iso_639_1,name
pelicula_id,,
862,en,English
8844,en,English
8844,fr,Français
15602,en,English
31357,en,English


In [188]:
movies_languages_normalize.reset_index(inplace=True)
movies_languages_normalize.head()

,pelicula_id,iso_639_1,name
0,862,en,English
1,8844,en,English
2,8844,fr,Français
3,15602,en,English
4,31357,en,English


In [189]:
movies_languages_normalize.rename(columns= {'name':'idioma'}, inplace=True)
movies_languages_normalize.head()

,pelicula_id,iso_639_1,idioma
0,862,en,English
1,8844,en,English
2,8844,fr,Français
3,15602,en,English
4,31357,en,English


## HASTA AQUI TENEMOS NORMALIZADAS LAS COLUMNAS DE belongs_to_collection, genres, production_companies, production_countries y spoker_languages

# Ahora a unir los df normalizados

In [190]:
movies_nor = pd.merge(movies_collection_normalize, movies_genres_normalize, on = 'pelicula_id')
movies_nor.head()

,pelicula_id,franquicia_id,franquicia,genero_id,nombre_genero
0,862,10194.0,Toy Story Collection,16.0,Animation
1,862,10194.0,Toy Story Collection,35.0,Comedy
2,862,10194.0,Toy Story Collection,10751.0,Family
3,15602,119050.0,Grumpy Old Men Collection,10749.0,Romance
4,15602,119050.0,Grumpy Old Men Collection,35.0,Comedy


In [191]:
movies_nor1 = pd.merge(movies_nor, movies_production_normalize, on = 'pelicula_id')
movies_nor2 = pd.merge(movies_nor1, movies_countries_normalize, on = 'pelicula_id')
movies_nor3 = pd.merge(movies_nor2, movies_languages_normalize, on = 'pelicula_id')
movies_nor3.head()

,pelicula_id,franquicia_id,franquicia,genero_id,nombre_genero,compañia,compañia_id,iso_3166_1,pais,iso_639_1,idioma
0,862,10194.0,Toy Story Collection,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
1,862,10194.0,Toy Story Collection,35.0,Comedy,Pixar Animation Studios,3.0,US,United States of America,en,English
2,862,10194.0,Toy Story Collection,10751.0,Family,Pixar Animation Studios,3.0,US,United States of America,en,English
3,15602,119050.0,Grumpy Old Men Collection,10749.0,Romance,Warner Bros.,6194.0,US,United States of America,en,English
4,15602,119050.0,Grumpy Old Men Collection,10749.0,Romance,Lancaster Gate,19464.0,US,United States of America,en,English


In [192]:
movies.rename(columns={'budget' : 'presupuesto', 
                       'id' : 'pelicula_id',
                       'original_language' : 'idioma_original',
                       'original_title' : 'titulo_original',
                       'popularity' : 'popularidad',
                       'release_date' : 'fecha_realizacion',
                       'revenue' : 'recaudacion',
                       'runtime' : 'duracion',
                       'title' : 'titulo',
                       'vote_average' : 'puntaje_reseña',
                       'vote_count' : 'votos_recibidos'}, inplace=True)
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   presupuesto            45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   pelicula_id            45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   idioma_original        45455 non-null  object 
 8   titulo_original        45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularidad            45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  fecha_realizacion      45379 non-null  object 
 15  re

In [193]:
movies_normalizado = pd.merge(movies_nor3, movies, on = 'pelicula_id')
movies_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57529 entries, 0 to 57528
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pelicula_id            57529 non-null  object 
 1   franquicia_id          57529 non-null  float64
 2   franquicia             57529 non-null  object 
 3   genero_id              57386 non-null  float64
 4   nombre_genero          57386 non-null  object 
 5   compañia               55159 non-null  object 
 6   compañia_id            55159 non-null  float64
 7   iso_3166_1             56917 non-null  object 
 8   pais                   56917 non-null  object 
 9   iso_639_1              57360 non-null  object 
 10  idioma                 57360 non-null  object 
 11  adult                  57529 non-null  object 
 12  belongs_to_collection  57529 non-null  object 
 13  presupuesto            57529 non-null  object 
 14  genres                 57529 non-null  object 
 15  ho

In [194]:
movies_normalizado.drop(['belongs_to_collection', 'homepage', 'poster_path', 'overview', 'video'], axis=1, inplace=True)
movies_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57529 entries, 0 to 57528
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   pelicula_id           57529 non-null  object 
 1   franquicia_id         57529 non-null  float64
 2   franquicia            57529 non-null  object 
 3   genero_id             57386 non-null  float64
 4   nombre_genero         57386 non-null  object 
 5   compañia              55159 non-null  object 
 6   compañia_id           55159 non-null  float64
 7   iso_3166_1            56917 non-null  object 
 8   pais                  56917 non-null  object 
 9   iso_639_1             57360 non-null  object 
 10  idioma                57360 non-null  object 
 11  adult                 57529 non-null  object 
 12  presupuesto           57529 non-null  object 
 13  genres                57529 non-null  object 
 14  imdb_id               57512 non-null  object 
 15  idioma_original    

In [195]:
movies_normalizado.drop(['idioma', 'titulo', 'adult', 'votos_recibidos', 'production_companies','production_countries'], axis=1, inplace=True)
movies_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57529 entries, 0 to 57528
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pelicula_id        57529 non-null  object 
 1   franquicia_id      57529 non-null  float64
 2   franquicia         57529 non-null  object 
 3   genero_id          57386 non-null  float64
 4   nombre_genero      57386 non-null  object 
 5   compañia           55159 non-null  object 
 6   compañia_id        55159 non-null  float64
 7   iso_3166_1         56917 non-null  object 
 8   pais               56917 non-null  object 
 9   iso_639_1          57360 non-null  object 
 10  presupuesto        57529 non-null  object 
 11  genres             57529 non-null  object 
 12  imdb_id            57512 non-null  object 
 13  idioma_original    57529 non-null  object 
 14  titulo_original    57529 non-null  object 
 15  popularidad        57529 non-null  object 
 16  fecha_realizacion  575

In [196]:
movies_normalizado.drop(['genres', 'tagline'], axis=1, inplace=True)

In [197]:
movies_normalizado.drop(['spoken_languages', 'status'], axis=1, inplace=True)
movies_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57529 entries, 0 to 57528
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pelicula_id        57529 non-null  object 
 1   franquicia_id      57529 non-null  float64
 2   franquicia         57529 non-null  object 
 3   genero_id          57386 non-null  float64
 4   nombre_genero      57386 non-null  object 
 5   compañia           55159 non-null  object 
 6   compañia_id        55159 non-null  float64
 7   iso_3166_1         56917 non-null  object 
 8   pais               56917 non-null  object 
 9   iso_639_1          57360 non-null  object 
 10  presupuesto        57529 non-null  object 
 11  imdb_id            57512 non-null  object 
 12  idioma_original    57529 non-null  object 
 13  titulo_original    57529 non-null  object 
 14  popularidad        57529 non-null  object 
 15  fecha_realizacion  57522 non-null  object 
 16  recaudacion        575

In [198]:
movies_duplicadas = movies_normalizado.duplicated().sum()
print(movies_duplicadas)

3162


### Dado que hay filas duplicadas, es necesario eliminarlas.

In [199]:
movies_normalizado = movies_normalizado.drop_duplicates()
movies_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54367 entries, 0 to 57528
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pelicula_id        54367 non-null  object 
 1   franquicia_id      54367 non-null  float64
 2   franquicia         54367 non-null  object 
 3   genero_id          54224 non-null  float64
 4   nombre_genero      54224 non-null  object 
 5   compañia           52679 non-null  object 
 6   compañia_id        52679 non-null  float64
 7   iso_3166_1         53755 non-null  object 
 8   pais               53755 non-null  object 
 9   iso_639_1          54198 non-null  object 
 10  presupuesto        54367 non-null  object 
 11  imdb_id            54350 non-null  object 
 12  idioma_original    54367 non-null  object 
 13  titulo_original    54367 non-null  object 
 14  popularidad        54367 non-null  object 
 15  fecha_realizacion  54360 non-null  object 
 16  recaudacion        543

In [200]:
movies_normalizado.drop(['imdb_id'], axis=1, inplace=True)

In [201]:
movies_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54367 entries, 0 to 57528
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pelicula_id        54367 non-null  object 
 1   franquicia_id      54367 non-null  float64
 2   franquicia         54367 non-null  object 
 3   genero_id          54224 non-null  float64
 4   nombre_genero      54224 non-null  object 
 5   compañia           52679 non-null  object 
 6   compañia_id        52679 non-null  float64
 7   iso_3166_1         53755 non-null  object 
 8   pais               53755 non-null  object 
 9   iso_639_1          54198 non-null  object 
 10  presupuesto        54367 non-null  object 
 11  idioma_original    54367 non-null  object 
 12  titulo_original    54367 non-null  object 
 13  popularidad        54367 non-null  object 
 14  fecha_realizacion  54360 non-null  object 
 15  recaudacion        54367 non-null  float64
 16  duracion           543

In [202]:
#movies_normalizado.to_csv('movies_normalizado.csv')

In [203]:
#credit_normalizado.to_csv('credit_normalizado.csv')

In [204]:
nulos_presupuesto = movies_normalizado["presupuesto"].isnull().sum()
nulos_recaudacion = movies_normalizado["recaudacion"].isnull().sum()
print(nulos_recaudacion, nulos_presupuesto)
## no hay nulos

0 0


In [205]:
nulos_fecha_ralizacion = movies_normalizado["fecha_realizacion"].isnull().sum()
nulos_fecha_ralizacion

7

In [206]:
# Los nulos en la columna fecha_realizacion deben eliminarse:

movies_normalizado = movies_normalizado.dropna(subset=['fecha_realizacion'])

In [207]:
# Creo la columna año_realizacion:
movies_normalizado['año_realizacion'] = pd.to_datetime(movies_normalizado['fecha_realizacion']).dt.year
movies_normalizado.head(3)

,pelicula_id,franquicia_id,franquicia,genero_id,nombre_genero,compañia,compañia_id,iso_3166_1,pais,iso_639_1,presupuesto,idioma_original,titulo_original,popularidad,fecha_realizacion,recaudacion,duracion,puntaje_reseña,año_realizacion
0,862,10194.0,Toy Story Collection,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,30000000,en,Toy Story,21.946943,1995-10-30,373554033.0,81.0,7.7,1995
1,862,10194.0,Toy Story Collection,35.0,Comedy,Pixar Animation Studios,3.0,US,United States of America,en,30000000,en,Toy Story,21.946943,1995-10-30,373554033.0,81.0,7.7,1995
2,862,10194.0,Toy Story Collection,10751.0,Family,Pixar Animation Studios,3.0,US,United States of America,en,30000000,en,Toy Story,21.946943,1995-10-30,373554033.0,81.0,7.7,1995


# Al observar el tipo de dato de las columnas presupuesto y recaudacion noto que presupuesto es 'int64'. Por lo que necesito cambiar a tipo float64 para poder calcular la ganancia.

Una vez hecha la transformacion, procedo a calcular la columna ganancia.

In [208]:
movies_normalizado['presupuesto'] = movies_normalizado['presupuesto'].astype('float64')

movies_normalizado['ganancia'] = movies_normalizado['recaudacion'] / movies_normalizado['presupuesto']
movies_normalizado

,pelicula_id,franquicia_id,franquicia,genero_id,nombre_genero,compañia,compañia_id,iso_3166_1,pais,iso_639_1,presupuesto,idioma_original,titulo_original,popularidad,fecha_realizacion,recaudacion,duracion,puntaje_reseña,año_realizacion,ganancia
0,862,10194.0,Toy Story Collection,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,30000000.0,en,Toy Story,21.946943,1995-10-30,373554033.0,81.0,7.7,1995,12.451801
1,862,10194.0,Toy Story Collection,35.0,Comedy,Pixar Animation Studios,3.0,US,United States of America,en,30000000.0,en,Toy Story,21.946943,1995-10-30,373554033.0,81.0,7.7,1995,12.451801
2,862,10194.0,Toy Story Collection,10751.0,Family,Pixar Animation Studios,3.0,US,United States of America,en,30000000.0,en,Toy Story,21.946943,1995-10-30,373554033.0,81.0,7.7,1995,12.451801
3,15602,119050.0,Grumpy Old Men Collection,10749.0,Romance,Warner Bros.,6194.0,US,United States of America,en,0.0,en,Grumpier Old Men,11.7129,1995-12-22,0.0,101.0,6.5,1995,NaN
4,15602,119050.0,Grumpy Old Men Collection,10749.0,Romance,Lancaster Gate,19464.0,US,United States of America,en,0.0,en,Grumpier Old Men,11.7129,1995-12-22,0.0,101.0,6.5,1995,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57524,460135,477208.0,DC Super Hero Girls Collection,16.0,Animation,Warner Bros. Animation,2785.0,US,United States of America,en,0.0,en,LEGO DC Super Hero Girls: Brain Drain,8.413734,2017-08-30,0.0,0.0,10.0,2017,NaN
57525,460135,477208.0,DC Super Hero Girls Collection,16.0,Animation,Warner Bros. Animation,2785.0,US,United States of America,pl,0.0,en,LEGO DC Super Hero Girls: Brain Drain,8.413734,2017-08-30,0.0,0.0,10.0,2017,NaN
57526,460135,477208.0,DC Super Hero Girls Collection,16.0,Animation,Warner Bros. Animation,2785.0,US,United States of America,pt,0.0,en,LEGO DC Super Hero Girls: Brain Drain,8.413734,2017-08-30,0.0,0.0,10.0,2017,NaN
57527,182030,200641.0,Red Lotus Collection,28.0,Action,Shaw Brothers,5798.0,NaN,NaN,zh,0.0,zh,火燒紅蓮寺之鴛鴦劍俠,0.674934,1965-12-22,0.0,88.0,5.6,1965,NaN


In [209]:
movies_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54360 entries, 0 to 57528
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pelicula_id        54360 non-null  object 
 1   franquicia_id      54360 non-null  float64
 2   franquicia         54360 non-null  object 
 3   genero_id          54218 non-null  float64
 4   nombre_genero      54218 non-null  object 
 5   compañia           52673 non-null  object 
 6   compañia_id        52673 non-null  float64
 7   iso_3166_1         53749 non-null  object 
 8   pais               53749 non-null  object 
 9   iso_639_1          54191 non-null  object 
 10  presupuesto        54360 non-null  float64
 11  idioma_original    54360 non-null  object 
 12  titulo_original    54360 non-null  object 
 13  popularidad        54360 non-null  object 
 14  fecha_realizacion  54360 non-null  object 
 15  recaudacion        54360 non-null  float64
 16  duracion           543

In [211]:
movies_normalizado = movies_normalizado.drop(columns=['franquicia_id', 'genero_id', 'compañia_id', 'iso_3166_1', 
                                                  'iso_639_1', 'popularidad', 'puntaje_reseña'])
movies_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54360 entries, 0 to 57528
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pelicula_id        54360 non-null  object 
 1   franquicia         54360 non-null  object 
 2   nombre_genero      54218 non-null  object 
 3   compañia           52673 non-null  object 
 4   pais               53749 non-null  object 
 5   presupuesto        54360 non-null  float64
 6   idioma_original    54360 non-null  object 
 7   titulo_original    54360 non-null  object 
 8   fecha_realizacion  54360 non-null  object 
 9   recaudacion        54360 non-null  float64
 10  duracion           54347 non-null  float64
 11  año_realizacion    54360 non-null  int64  
 12  ganancia           35401 non-null  float64
dtypes: float64(4), int64(1), object(8)
memory usage: 5.8+ MB


In [212]:
movies_normalizado = movies_normalizado.drop_duplicates()
movies_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34826 entries, 0 to 57528
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pelicula_id        34826 non-null  object 
 1   franquicia         34826 non-null  object 
 2   nombre_genero      34737 non-null  object 
 3   compañia           33445 non-null  object 
 4   pais               34297 non-null  object 
 5   presupuesto        34826 non-null  float64
 6   idioma_original    34826 non-null  object 
 7   titulo_original    34826 non-null  object 
 8   fecha_realizacion  34826 non-null  object 
 9   recaudacion        34826 non-null  float64
 10  duracion           34813 non-null  float64
 11  año_realizacion    34826 non-null  int64  
 12  ganancia           21790 non-null  float64
dtypes: float64(4), int64(1), object(8)
memory usage: 3.7+ MB


In [218]:
movies_normalizado['pelicula_id'] = movies_normalizado['pelicula_id'].astype(np.int64)

# Ahora terminaremos de normalizar y reducir el dataset de credit a los efectos teniendo en mira las funciones a desarrollar.

In [225]:
crew_normalizado = pd.read_csv('directores.csv')
crew_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34344083 entries, 0 to 34344082
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Unnamed: 0   int64  
 1   pelicula_id  int64  
 2   puesto       object 
 3   crew_nombre  object 
 4   presupuesto  float64
 5   recaudacion  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 1.5+ GB


In [ ]:
mvp_funciones = pd.merge(crew_normalizado[crew_normalizado['puesto'] == 'Director'], movies_normalizado, on='pelicula_id')
mvp_funciones.info()

In [ ]:
mvp_funciones['ganancia'] = mvp_funciones['ganancia'].fillna(0)

In [ ]:
mvp_funciones= mvp_funciones.drop(columns=['Unnamed: 0', 'presupuesto_x', 'recaudacion_x', 'nombre_genero'])
mvp_funciones.info()

In [ ]:
mvp_funciones = mvp_funciones.drop_duplicates()
mvp_funciones.info()

In [ ]:
mvp_funciones.to_csv('dataset_normalizado.csv')